In [1]:
#default_exp read
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from fastcore.xtras import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Reading
> Este módulo concentra funções para a leitura dos diversos arquivos que compões a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.

In [3]:
#export
from typing import Union, Tuple
from pathlib import Path

import pandas as pd
from pyarrow import ArrowInvalid


from anateldb.query import update_mosaico, update_stel, update_radcom, update_base
from anateldb.merge import aero_common, aero_new

## MOSAICO
> O mosaico atualmente é composto por 3 bases complementares originárias de um banco `MongoDB`: `Estações, Plano Básico e Histórico`. Os módulos aqui leem, processam e mesclam as duas primeiras.

In [4]:
#export
def read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError):
        file = Path(f"{folder}/{stem}.fth")
        try:
            df = pd.read_feather(file)
        except (ArrowInvalid, FileNotFoundError):
            file = Path(f"{folder}/{stem}.xlsx")
            try:
                df = pd.read_excel(file, engine="openpyxl", sheet_name="DataBase")
            except Exception as e:
                raise ValueError(f"Error when reading {file}") from e
    return df


def read_mosaico(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Lê o banco de dados salvo localmente do MOSAICO e opcionalmente o atualiza."""
    return update_mosaico(folder) if update else read_df(folder, "mosaico")

In [5]:
folder = Path.cwd().parent / 'dados' / 'tabular'

In [ ]:
import pandas_read_xml as pdx
df = pdx.read_xml(f'{folder})

In [17]:
mos = read_mosaico(folder, True)
mos.info()

🕛  Baixando e consolidando os dados do Mosaico...

KeyError: 'Id'

In [ ]:
import pandas_read_xml as pdx
from unidecode import unidecode
from anateldb.format import *
from anateldb.query import _read_estações
df = _read_estações (f'{folder}/estações.zip')

In [ ]:
df.columns

Index(['item', 'siglaservico', 'id', 'state', 'entidade', 'fistel', 'cnpj',
       'codmunicipio', 'municipio', 'uf', 'entidade', 'administrativo',
       'enderecos', 'estacao_principal', 'estacao_auxiliar',
       'transmissor_auxiliar', 'transmissor_auxiliar2', 'linha_auxiliar',
       'antena_auxiliar', 'horario_funcionamento', 'rds'],
      dtype='object')

## STEL & RADCOM

In [ ]:
#export
def read_stel(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    return update_stel(folder) if update else read_df(folder, "stel")


def read_radcom(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01 caso `update = True` ou não exista o arquivo local"""
    return update_radcom(folder) if update else read_df(folder, "radcom")

## Bases Externas da Aeronáutica

In [ ]:
#export
def read_icao(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Lê a base de dados do Frequency Finder e Canalização VOR/ILS/DME"""
    if update:
        # TODO: atualizar a base de dados do Frequency Finder e Canalização VOR/ILS/DME
        # update_icao(folder)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    return read_df(folder, "icao")


def read_aisw(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME
        # update_pmec(folder)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    return read_df(folder, "aisw")


def read_aisg(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    if update:
        # TODO: Atualizar a base de dados do GEOAISWEB
        # update_geo(folder)
        raise NotImplementedError(
            "Atualizar da base de dados do Frequency Finder e Canalização VOR/ILS/DME não implementado"
        )
    return read_df(folder, "aisg")


def read_aero(
    folder: Union[str, Path],
    up_icao: bool = False,
    up_aisw: bool = False,
    up_aisg: bool = False,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Lê os arquivos de dados da aeronáutico e retorna os registros comuns e únicos"""
    icao = read_icao(folder, up_icao)
    pmec = read_aisw(folder, up_aisw)
    geo = read_aisg(folder, up_aisg)
    icao["Description"] = icao.Description.astype("string")
    pmec["Description"] = pmec.Description.astype("string")
    geo["Description"] = geo.Description.astype("string")
    common = aero_common(icao, pmec, geo)
    new = aero_new(icao, pmec, geo)
    return common, new

## Base Consolidada

In [ ]:
#export
def read_base(folder: Union[str, Path], update: bool = False) -> pd.DataFrame:
    """Lê a base de dados e opcionalmente a atualiza antes da leitura"""
    return update_base(folder) if update else read_df(folder, "base")